In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import scipy.signal
import scipy.stats

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'damselfly/data/datasets')
SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')

"""
Date: 6/25/2021
Description: template
"""


def shift_and_sum(signal, x_range, y_range, N):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    x_antenna = x_antenna.reshape((1, x_antenna.size)).repeat(N ** 2, axis = 0)
    y_antenna = y_antenna.reshape((1, y_antenna.size)).repeat(N ** 2, axis = 0)
    
    x_array = np.linspace(-x_range, x_range, N)
    y_array = np.linspace(-y_range, y_range, N)
    
    x_grid, y_grid = np.meshgrid(x_array, y_array)
    x_grid = x_grid.flatten().reshape((x_grid.size, 1)).repeat(nch, axis=-1)
    y_grid = y_grid.flatten().reshape((y_grid.size, 1)).repeat(nch, axis=-1)
    
    #print(x_antenna.shape, x_grid.shape)
    
    phase_grid = 2 * np.pi * np.sqrt((x_grid - x_antenna) ** 2 + (y_grid - y_antenna) ** 2) / wavelength_lo
    
    anti_spiral = angles.reshape((1, angles.size)).repeat(N**2, axis=0)
    
    summed_signals = np.matmul(np.exp(1j * (phase_grid-anti_spiral)), signal)
    
    summed_signals = summed_signals.reshape((N, N, summed_signals.shape[-1]))
    
    return summed_signals


def shift_signal(signal, x_new, y_new):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    x_antenna = x_antenna.reshape((x_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    y_antenna = y_antenna.reshape((y_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    
    d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    
    
    
    phase_shift = 2 * np.pi * (d_new - d_old) / wavelength_lo
    #print(phase_shift)
    
    shifted_signal = np.exp(-1j * phase_shift) * signal
    
    return shifted_signal

def shift_pc(pc, x_new, y_new):
    
    nch = 60
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    
    shifted_pc = np.zeros(nch * pc.size, np.complex64)
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)

    d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    

    phase_shift = 2 * np.pi * (d_new - d_old) / wavelength_lo + angles
    
    for i, phase in enumerate(phase_shift):
        shifted_pc[i * pc.size:(i + 1) * pc.size] = np.exp(-1j * phase) * pc
    

    return shifted_pc

def tile_pc(pc):
    
    nch = 60
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    #r_array = 0.10
    #wavelength_lo = 3e8 / 25.86e9
    
    tile_pc = np.zeros(nch * pc.size, np.complex64)
    
    phase_shift = angles
    
    for i, phase in enumerate(phase_shift):
        tile_pc[i * pc.size:(i + 1) * pc.size] = np.exp(-1j * phase) * pc
    

    return tile_pc


In [ ]:
os.listdir(os.path.join(PATH, 'mayfly/data/datasets'))

In [ ]:
os.listdir(os.path.join(PATH, 'damselfly/data/sim_data'))

# load pc's

In [ ]:
#pca_all = np.load(os.path.join(PATH, 'damselfly/data', '210914_frequency_spectra_imag_principle_components.npz'))
#pca_all_evals = np.load(os.path.join(PATH, 'damselfly/data', '211007_principle_components_all_evals.npy'))
#pca_all_evecs = np.load(os.path.join(PATH, 'damselfly/data', '211007_principle_components_all_evecs.npy'))

pca_range_evals = np.load(os.path.join(PATH, 'damselfly/data', '211009_principle_components_1ev_energy_range_evals.npy'))
pca_range_evecs = np.load(os.path.join(PATH, 'damselfly/data', '211009_principle_components_1ev_energy_range_evecs.npy'))

In [ ]:
pca_range_evals.shape

# load data

In [ ]:
data = os.path.join(os.path.join(PATH, 'mayfly/data/datasets'), '211002_mf_84_100_slice8192.h5')
#data = os.path.join(os.path.join(PATH, 'damselfly/data/sim_data'), '210729_df_84_1slice.h5')

h5datafile = h5py.File(data, 'r')

energies = h5datafile['meta']['energy'][:]

select_energies = np.argwhere(abs(energies - 18550) <= 0.5).squeeze()

In [ ]:
data = os.path.join(os.path.join(PATH, 'mayfly/data/datasets'), '211002_mf_84_100_slice8192.h5')
#data = os.path.join(os.path.join(PATH, 'damselfly/data/sim_data'), '210729_df_84_1slice.h5')

h5datafile = h5py.File(data, 'r')

print(h5datafile['data'].shape)

Nsignal = h5datafile['data'].shape[0]

#Nsignal = len(list(h5datafile['signal'].keys()))
Nsample = 8192
Nch = 60

dataset = np.zeros((select_energies.size, Nch * Nsample), dtype=np.complex64)


for i, index in enumerate(select_energies):
#    dataset[i, :] = np.fft.fftshift(np.fft.fft(h5datafile['data'][i, :].reshape(60, 8192).sum(axis=0))) / 8192
#    dataset[i, :] = h5datafile['signal'][f'{i}'][0, :]


    #fig = plt.figure(figsize=(13,8))
    #ax = fig.add_subplot(1,1,1)
    
    #ax.plot(h5datafile['data'][index, :].real)
    #ax.plot(h5datafile['data'][index, :].imag)
    
    #ax.set_xlim(0, 1024)
    #plt.show()
    #input()
    
    #signal = h5datafile['data'][index, :]
    
    #template = signal / np.sqrt(var * np.vdot(signal, signal))
    
    #print(abs(np.vdot(template, signal)))
    
    #signal = (np.fft.fftshift(np.fft.fft(h5datafile['data'][index, :].reshape(60, 8192), axis=-1), axes=-1) / 8192).flatten()
    
    #template = signal / np.sqrt(var * np.vdot(signal, signal) / 8192)
    
    #print(abs(np.vdot(template, signal)))
    
    
    
    dataset[i, :] = (np.fft.fftshift((np.fft.fft(h5datafile['data'][index, :].reshape(60, 8192), axis=-1)), axes=-1) / 8192).flatten()

In [ ]:
plt.plot(dataset[0, :].reshape((60, 8192)).sum(axis=0).real)
plt.plot(dataset[0, :].reshape((60, 8192)).sum(axis=0).imag)


plt.plot(abs(dataset[0, :].reshape((60, 8192)).sum(axis=0)))

print(abs(dataset[0, :].reshape((60, 8192)).sum(axis=0)).sum())

In [ ]:
plt.plot((anti_spiral * dataset[0, :].reshape((60, 8192))).sum(axis=0).real)
plt.plot((anti_spiral * dataset[0, :].reshape((60, 8192))).sum(axis=0).imag)

plt.plot(abs((anti_spiral * dataset[0, :].reshape((60, 8192))).sum(axis=0)))
print(abs((anti_spiral * dataset[0, :].reshape((60, 8192))).sum(axis=0)).sum())

In [ ]:
k = 4

print(abs((dataset[k, :].reshape((60, 8192)).sum(axis=0))).sum() ** 2)


anti_spiral = (np.arange(0, 60, 1) * 2 * np.pi / 60).reshape((60, 1)).repeat(8192, axis=-1)
anti_spiral = np.exp(-1j * anti_spiral)

print(abs((anti_spiral * dataset[k, :].reshape((60, 8192))).sum(axis=0)).sum() ** 2)

# select pc's

In [ ]:
pc_max_vals = abs(pca_range_evecs).max(axis=1)

In [ ]:
selected_pc_inds = np.argwhere(pc_max_vals >= 0.75)

select_pc = pca_range_evecs[selected_pc_inds[:, 0], :, selected_pc_inds[:, 1]]

print(select_pc.shape)


In [ ]:
n = 0
m = 0

pc = select_pc[n, :]
signal = dataset[m, :]

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(signal.real)
ax.plot(signal.imag)

tiled_pc = tile_pc(pc)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(tiled_pc.real)
ax.plot(tiled_pc.imag)




# correlation using unsummed signal and repeated pc

In [ ]:

n = 0
m = 0

pc = select_pc[n, :]
signal = dataset[m, :]

tiled_pc = tile_pc(pc)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(tiled_pc))
ax.plot(abs(pc))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(signal))
ax.plot(abs(pc) / 1e9)

In [ ]:

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(tiled_pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

ax.plot(abs(circ_conv))
ax.plot(abs(signal))
#ax.set_xlim(0, 5*8192)

# shift signal to x=1cm, correlate with unshifted pc

In [ ]:
n = 0
m = 0

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.01, 0.0).flatten()

tiled_pc = tile_pc(pc)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(tiled_pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('amplitude goes down')


# shift signal to x=1cm, correlate with shifted pc

In [ ]:
n = 11
m = 12

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.01, 0.0).flatten()

pc = shift_pc(pc, 0.01, 0.0)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('amplitude back to previous peak, asymmetric convolution')


# shift signal to y=1cm, correlate with shifted pc to x=1cm

In [ ]:
n = 0
m = 0

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.00, 0.01).flatten()

pc = shift_pc(pc, 0.01, 0.00)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('different angular position just circular shifts the convolution')

# signal auto-correlation

In [ ]:
n = 2
m = 0

#pc = select_pc[n, :]
signal = dataset[m, :]

norm_signal = signal / np.sqrt((abs(signal) ** 2).sum())

#signal = shift_signal(signal, 0.00, 0.01).flatten()

#pc = shift_pc(pc, 0.01, 0.00)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(signal)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))

# figure out noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50 / 8192
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
#noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(noise))
ax.plot(abs(signal))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(scipy.signal.fftconvolve(noise + signal, signal)))

# signal auto-correlation with noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50 / 8192
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
#noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 2
m = 8

#pc = select_pc[n, :]
signal = dataset[m, :]

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(signal))
ax.set_xlim(0, 8192)

#print(abs(1 / np.sqrt(var * np.vdot(signal, signal))))

template = signal * 1 / np.sqrt(var * np.vdot(signal, signal))

#print(abs(np.vdot(signal, signal)) / 50)
print(abs(np.vdot(template, signal)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(template))
ax.set_xlim(0, 8192)

#signal = shift_signal(signal, 0.00, 0.01).flatten()

#pc = shift_pc(pc, 0.01, 0.00)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(scipy.signal.fftconvolve(signal + noise, signal))
#print(np.max(circ_conv))

#print(np.vdot(signal, signal))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))

circ_conv = abs(np.fft.ifft(np.fft.fft(signal + noise) * np.fft.fft(signal)))
#print(np.max(circ_conv))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))

circ_conv = abs(np.fft.ifft(np.fft.fft(noise) * np.fft.fft(signal.conjugate())))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))

# correlation using unsummed signal and repeated pc, add noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50 / 8192
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
#noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 10
m = 8

pc = select_pc[n, :]
signal = dataset[m, :]

tiled_pc = tile_pc(pc)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal + noise)) * np.fft.fft(tiled_pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
print(np.max(circ_conv))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(noise)) * np.fft.fft(tiled_pc)))
ax.plot(abs(circ_conv))

print(np.max(circ_conv))

# shift signal to x=1cm, correlate with unshifted pc, add noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 2
m = 2

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.01, 0.0).flatten()

tiled_pc = tile_pc(pc)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal + noise)) * np.fft.fft(tiled_pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('amplitude goes down')


# shift signal to x=1cm, correlate with shifted pc, add noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50 / 8192
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
#noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 2
m = 2

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.01, 0.0).flatten()

pc = shift_pc(pc, 0.01, 0.0)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal + noise)) * np.fft.fft(pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('amplitude back to previous peak, asymmetric convolution')


# shift signal to y=1cm, correlate with shifted pc, add noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 0
m = 1

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.00, 0.01).flatten()

pc = shift_pc(pc, 0.01, 0.00)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal + noise)) * np.fft.fft(pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('different angular position just circular shifts the convolution')